# Simple Reinforcement Learning in Tensorflow

In [2]:
import tensorflow.compat.v1 as tf
import numpy as np
import warnings

warnings.filterwarnings('ignore')

## своего рода окружение

In [3]:
#список наших бандитов. Бандит №4 наиболее оптимален для выбора
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #генерим случайное число
    result = np.random.randn(1)
    if result > bandit:
        #победа
        return 1
    else:
        #лохпидор
        return -1

## своего рода агент (состоит из набора значений для бандитов)

In [4]:
tf.reset_default_graph()
tf.disable_eager_execution()# вот тут сделала плохую вещь, так не надо, а то может быть не жоск типировано



#оформляем feed-forward часть нейросети и происходит выбор действия
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#тута нейросеть принимает на вход действие и его результат, чтобы оценить функцию потерь и обновить веса сети
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

## околообучение

In [5]:
total_episodes = 1000 #кол-во итераций обучения
total_reward = np.zeros(num_bandits) #обозначаем начальный выигрыш всех бандитов равен 0
e = 0.1 #вер-ть случайного выбора
init = tf.global_variables_initializer()

#Запускаем граф tensorflow
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #выбираем действие либо случайно либо на основе нейросети
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        #получаем результат, выбрав одного 
        reward = pullBandit(bandits[action]) 
        
        #обновляем веса
        _,resp,ww = sess.run([update,responsible_weight,weights], 
                      feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #обновляем общий выигрыш каждого бандита
        total_reward[action] += reward
        if i % 50 == 0:
            print("Общий выигрыш бандитов сейчас равен " + str(num_bandits) + 
            " bandits: " + str(total_reward))
        i+=1
print("Агент думает(Саймон говорит), лучший номер " + str(np.argmax(ww)+1))
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("по факту")
else:
    print("чет не то сделали")

2024-05-15 02:10:46.000962: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-15 02:10:46.001042: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-15 02:10:46.001080: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2024-05-15 02:10:46.002326: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Общий выигрыш бандитов сейчас равен 4 bandits: [-1.  0.  0.  0.]
Общий выигрыш бандитов сейчас равен 4 bandits: [-1. -2. -2. 28.]
Общий выигрыш бандитов сейчас равен 4 bandits: [-1. -3. -1. 74.]
Общий выигрыш бандитов сейчас равен 4 bandits: [ -1.  -4.  -2. 120.]
Общий выигрыш бандитов сейчас равен 4 bandits: [ -2.  -4.  -3. 168.]
Общий выигрыш бандитов сейчас равен 4 bandits: [ -1.  -4.  -2. 214.]
Общий выигрыш бандитов сейчас равен 4 bandits: [  1.  -4.  -4. 258.]
Общий выигрыш бандитов сейчас равен 4 bandits: [  1.  -4.  -3. 303.]
Общий выигрыш бандитов сейчас равен 4 bandits: [  2.  -3.  -4. 350.]
Общий выигрыш бандитов сейчас равен 4 bandits: [  1.  -2.  -4. 398.]
Общий выигрыш бандитов сейчас равен 4 bandits: [  0.  -2.  -3. 446.]
Общий выигрыш бандитов сейчас равен 4 bandits: [ -2.  -4.  -1. 488.]
Общий выигрыш бандитов сейчас равен 4 bandits: [ -1.  -6.   1. 531.]
Общий выигрыш бандитов сейчас равен 4 bandits: [ -2.  -7.   1. 575.]
Общий выигрыш бандитов сейчас равен 4 bandits: